In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os, pickle
from time import time
FOLDER_PATH = 'drive/My Drive/ML/colab'

In [2]:
test_set = pickle.load(open(os.path.join(FOLDER_PATH,'pickle','test_set.p'),'rb'))
order_product_prior = pd.read_csv(os.path.join(FOLDER_PATH,'data/order_products_prior.csv'))

In [3]:
order_product_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [4]:
product_purchase_count = order_product_prior.groupby('product_id').count()[['order_id']].reset_index().rename(columns={"order_id":"qty"})
product_purchase_count.head()

,product_id,qty
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [5]:
top10_product = product_purchase_count.sort_values('qty',ascending=False).product_id[:10].values

In [6]:
def cal_avg_precision(actual, predicted):
    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if num_hits == 0.0:
        return 0.0

    return score / num_hits

In [8]:
# 1. get what user new product and reordered product
# 2. get top 10 item that user didnt order
scores = []
start = time()
for u in test_set.user_id.unique():
    gt_product = test_set.query('user_id=={}'.format(u)).product_id.values
    score = cal_avg_precision(gt_product, top10_product)
    scores.append(score)
    if u % 5000 == 0:
        print(u, end=' | ')
print(time()-start)  

80000 | 65000 | 85000 | 50000 | 100000 | 15000 | 195000 | 135000 | 130000 | 90000 | 185000 | 190000 | 10000 | 25000 | 160000 | 20000 | 180000 | 170000 | 75000 | 140000 | 55000 | 145000 | 60000 | 105000 | 150000 | 165000 | 45000 | 155000 | 2367.6957659721375


In [10]:
np.mean(scores)

0.22083999492549972

In [9]:
import pickle

pickle.dump(scores, open('drive/My Drive/ML/colab/pickle/test_baseline_allitem.p','wb'))